In [1]:
import os
import json
import random
import string
import getpass
import hashlib
import tkinter as tk
from tkinter import messagebox, simpledialog, scrolledtext
from cryptography.fernet import Fernet

# File Paths
MASTER_HASH_FILE = "master.hash"
KEY_FILE = "secret.key"
PASSWORDS_FILE = "passwords.json"

# Master Password Functions
def setup_master_password():
    while True:
        master_password = simpledialog.askstring("Setup", "Set your master password:", show='*')
        confirm_password = simpledialog.askstring("Setup", "Confirm your master password:", show='*')
        if master_password != confirm_password:
            messagebox.showerror("Error", "Passwords do not match. Try again.")
        else:
            master_hash = hashlib.sha256(master_password.encode()).hexdigest()
            with open(MASTER_HASH_FILE, 'w') as file:
                file.write(master_hash)
            return master_password

def verify_master_password():
    if not os.path.exists(MASTER_HASH_FILE):
        return setup_master_password()
    while True:
        master_password = simpledialog.askstring("Login", "Enter your master password:", show='*')
        with open(MASTER_HASH_FILE, 'r') as file:
            stored_hash = file.read().strip()
        if hashlib.sha256(master_password.encode()).hexdigest() != stored_hash:
            messagebox.showerror("Error", "Incorrect master password. Try again.")
        else:
            return master_password

# Encryption Functions
def write_key():
    key = Fernet.generate_key()
    with open(KEY_FILE, 'wb') as key_file:
        key_file.write(key)

def load_key():
    if not os.path.exists(KEY_FILE):
        write_key()
    return open(KEY_FILE, 'rb').read()

def encrypt_message(message, key):
    return Fernet(key).encrypt(message.encode())

def decrypt_message(encrypted_message, key):
    return Fernet(key).decrypt(encrypted_message).decode()

# Password Management
def generate_password(length=12):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

def save_password(service, username, password, key):
    encrypted_password = encrypt_message(password, key)
    entry = {service: {'username': username, 'password': encrypted_password.decode()}}
    if os.path.exists(PASSWORDS_FILE):
        with open(PASSWORDS_FILE, 'r') as file:
            data = json.load(file)
    else:
        data = {}
    data.update(entry)
    with open(PASSWORDS_FILE, 'w') as file:
        json.dump(data, file, indent=4)
    messagebox.showinfo("Success", f"Password for '{service}' saved.")

def load_passwords(key):
    if not os.path.exists(PASSWORDS_FILE):
        return {}
    with open(PASSWORDS_FILE, 'r') as file:
        encrypted_data = json.load(file)
    return {s: {'username': d['username'], 'password': decrypt_message(d['password'].encode(), key)} for s, d in encrypted_data.items()}

def delete_password(service, key):
    if not os.path.exists(PASSWORDS_FILE):
        messagebox.showinfo("Info", "No passwords stored yet.")
        return
    with open(PASSWORDS_FILE, 'r') as file:
        data = json.load(file)
    if service in data:
        del data[service]
        with open(PASSWORDS_FILE, 'w') as file:
            json.dump(data, file, indent=4)
        messagebox.showinfo("Success", f"Password for '{service}' deleted.")
    else:
        messagebox.showerror("Error", f"No entry found for '{service}'.")

# GUI Functions
def show_passwords():
    passwords = load_passwords(key)
    window = tk.Toplevel()
    window.title("Stored Passwords")
    text_area = scrolledtext.ScrolledText(window, width=60, height=20)
    text_area.pack(padx=10, pady=10)
    if not passwords:
        text_area.insert(tk.END, "No passwords stored.")
    else:
        for service, creds in passwords.items():
            text_area.insert(tk.END, f"Service: {service}\nUsername: {creds['username']}\nPassword: {creds['password']}\n{'-'*50}\n")

def add_password():
    service = simpledialog.askstring("Add Password", "Enter service name:")
    username = simpledialog.askstring("Add Password", "Enter username:")
    password = simpledialog.askstring("Add Password", "Enter password (leave blank to generate):", show='*')
    if not password:
        password = generate_password()
        messagebox.showinfo("Generated Password", f"Generated password: {password}")
    save_password(service, username, password, key)

def delete_password_prompt():
    service = simpledialog.askstring("Delete Password", "Enter service name to delete:")
    if service:
        delete_password(service, key)

def generate_password_prompt():
    try:
        length = int(simpledialog.askstring("Generate Password", "Enter length:"))
    except (ValueError, TypeError):
        length = 12
    password = generate_password(length)
    messagebox.showinfo("Generated Password", f"Generated password: {password}")

# Main GUI Panel
def main_gui():
    root = tk.Tk()
    root.title("Password Manager")
    root.geometry("300x300")
    
    tk.Button(root, text="Add Password", command=add_password, width=25).pack(pady=5)
    tk.Button(root, text="View Stored Passwords", command=show_passwords, width=25).pack(pady=5)
    tk.Button(root, text="Generate Password", command=generate_password_prompt, width=25).pack(pady=5)
    tk.Button(root, text="Delete Password", command=delete_password_prompt, width=25).pack(pady=5)
    tk.Button(root, text="Exit", command=root.quit, width=25, bg="red").pack(pady=5)
    
    root.mainloop()

# Start Program
if __name__ == "__main__":
    verify_master_password()
    key = load_key()
    main_gui()
